In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')

In [2]:
from langchain_groq import ChatGroq


model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001AAD0286590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AAD0285600>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi my name is Anirudh ")])

AIMessage(content="Hello Anirudh, it's nice to meet you! 👋 \n\nWhat can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 17, 'total_tokens': 44, 'completion_time': 0.049090909, 'prompt_time': 7.69e-05, 'queue_time': 0.022645967, 'total_time': 0.049167809}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a4093588-6128-4ef6-ab5d-d2a150823f54-0', usage_metadata={'input_tokens': 17, 'output_tokens': 27, 'total_tokens': 44})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content='Hi my name is Anirudh'),
        AIMessage(content="Hello Anirudh! It's nice to meet you. \n\nWhat can I do for you today? 😊  \n\n"),
        HumanMessage(content='Hey what is my name?')
    ]
)

AIMessage(content='Your name is Anirudh, you told me earlier! 😄  \n\nIs there anything else I can help you with?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 58, 'total_tokens': 87, 'completion_time': 0.052727273, 'prompt_time': 0.001833202, 'queue_time': 0.020312906000000002, 'total_time': 0.054560475}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-81129f02-2c37-4ea1-a3f8-64fc5abb279f-0', usage_metadata={'input_tokens': 58, 'output_tokens': 29, 'total_tokens': 87})

In [5]:
## Message history
#It will store the inputs which was passed earlier to maintain a converstaional history

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [10]:
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()

    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [9]:
config={"configurable":{'session_id':'abcd'}}

In [12]:
response=with_message_history.invoke(
    [
        HumanMessage(content='Hi my name is Anirudh and i am a data scientist'),

    ],
    config=config
)

In [14]:
with_message_history.invoke(
    [
        HumanMessage(content='what is my name?'),

    ],
    config=config
)

#This it is able to remember because it is using same configuration which is having same conversational history

AIMessage(content="Your name is Anirudh.  \n\nI'm here to help, is there anything else you'd like to talk about?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 238, 'total_tokens': 268, 'completion_time': 0.054545455, 'prompt_time': 0.036650237, 'queue_time': 0.022045772999999998, 'total_time': 0.091195692}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-56f0f43e-5cc5-416d-a14a-c737aec90569-0', usage_metadata={'input_tokens': 238, 'output_tokens': 30, 'total_tokens': 268})

In [16]:
#Lets check chainging the config with different session_id
config2={"configurable":{'session_id':'mcc'}}

with_message_history.invoke(
    [
        HumanMessage(content='what is my name?'),

    ],
    config=config2
)
#It doesnt have the history of messages hence not giving us correct output

AIMessage(content="Since I don't have access to any information about you, including your name, I can't tell you what it is.  \n\nIs there anything else I can help you with? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 64, 'total_tokens': 108, 'completion_time': 0.08, 'prompt_time': 0.002301869, 'queue_time': 0.020424368, 'total_time': 0.082301869}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e73bfd0-8a21-4781-bb71-40235d8cc0dc-0', usage_metadata={'input_tokens': 64, 'output_tokens': 44, 'total_tokens': 108})

In [25]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",'You are a helpful assistant . Answer all the questions to the nest of your ability in {language}'),
        MessagesPlaceholder(variable_name='messages')
    ]
)

In [26]:
chain=prompt|model

In [28]:
chain.invoke({'messages':[HumanMessage(content='Hi my name is krish')],'language':'hindi'})

AIMessage(content='नमस्ते कृष्ण! 😊 \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। आप मुझसे कोई भी प्रश्न पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपको सबसे अच्छी जानकारी दे सकूँ।  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 33, 'total_tokens': 94, 'completion_time': 0.110909091, 'prompt_time': 0.001822441, 'queue_time': 0.019432487999999998, 'total_time': 0.112731532}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-913e3214-e30c-4e6f-95bc-e69155a3c699-0', usage_metadata={'input_tokens': 33, 'output_tokens': 61, 'total_tokens': 94})

In [29]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key='messages')

config={'configurable':{'session_id':'agc'}}

response=with_message_history.invoke({'messages':[HumanMessage(content='Hi my name is Anirudh')],'language':'hindi'},config=config)

In [30]:
response.content

'नमस्ते अनिरूद्ध! \n\nआपका नाम सुनकर बहुत अच्छा लगा। 😊  \n\nमैं आपकी मदद कैसे कर सकता हूँ?  \n'

In [ ]:
""" Manaing the conversational history because for a particular session if we keep on storing the history it will be out of bound to limit hence there 
is a rwquirement to limit the size of message you are passing in"""

"trim_message:helps to reduce how many messages we are sending to the model"

In [40]:
from langchain_core.messages import SystemMessage,trim_messages

def token_counter(messages):
    # Simple token count based on word count for illustration (replace with actual tokenization logic)
    return sum(len(message.content.split()) for message in messages)

# Define trimming function
trimmer = trim_messages(
    max_tokens=20,
    strategy='last',
    token_counter=token_counter,
    include_system=True,
    allow_partial=False,
    start_on='human'
)



messages=[
    SystemMessage(content='you are a good assistant'),
    HumanMessage(content='hi ! I am mini alexa'),
    AIMessage(content='Hi'),
    HumanMessage(content='WHAT IS 1+1'),
    AIMessage(content='2'),
    HumanMessage(content='I LIKE CHICKEN'),
    AIMessage(content='Nice'),
    HumanMessage(content='I was born on 14 august'),
    AIMessage(content='That is interesting'),
    HumanMessage(content='I love playing outdoor sports'),
    AIMessage(content='Nice ')
]

In [41]:
trimmer.invoke(messages)  #it took only less no of messags from history

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I was born on 14 august', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That is interesting', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love playing outdoor sports', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice ', additional_kwargs={}, response_metadata={})]

In [ ]:
#Connecting this trimmer to chain

from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer) 
    | prompt
    |model
)

chain.invoke({"messages":messages + [HumanMessage(content='what ice cream do i like?')],"language":'English'})

#So we are not getting because trimmer is removing that from history based on token size

AIMessage(content="As a helpful assistant, I don't have access to your personal information, including your ice cream preferences! \n\nWhat's your favorite flavor?  🍦😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 57, 'total_tokens': 96, 'completion_time': 0.070909091, 'prompt_time': 0.001832991, 'queue_time': 0.020515058, 'total_time': 0.072742082}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-caf20417-6514-46ba-bb80-d51242952825-0', usage_metadata={'input_tokens': 57, 'output_tokens': 39, 'total_tokens': 96})

In [44]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key='messages')
config={"configurable":{'session_id':'abscdd'}}


In [45]:
response=with_message_history.invoke({'messages':[HumanMessage(content='what ice cream do i eat')],'language':'hindi'},config=config)

In [46]:
response.content

'आपको किस तरह का आइसक्रीम पसंद है? 🤔\n\nमुझे बताएं तो मैं आपको कुछ सुझाव दे सकूँ! 🍨 \n\nक्या आपको कोई खास फ्लेवर पसंद है? (जैसे, चॉकलेट, स्ट्रॉबेरी, वनीला) 🍓🍫🍦 \nक्या आप कोई क्रंची टेक्सचर पसंद करते हैं? (जैसे, बटरबटर, नट्स) 🥜\nया फिर आप कोई लजीज और क्रीमी आइसक्रीम चाहते हैं? 🍦\n\n\nआपके पसंद के अनुसार मैं आपको सही आइसक्रीम सुझा सकता हूँ! 😊\n'